In [6]:
import pandas as pd
import altair as alt
from google.cloud import bigquery
import geopandas as gpd
from shapely import wkt
from census import Census
from us import states
#import google.auth
#from google.cloud import bigquery
#from google.cloud import bigquery_storage
bqclient = bigquery.Client.from_service_account_json('C:/Users/bennd/Documents/MUSA509/TransitPolicyApp-99838a65a6ed.json')
pd.set_option('mode.chained_assignment', None)
#ridership = pd.read_csv("data/ridership.csv")
#ridership['month'] = pd.to_datetime(ridership['month'])
covid = pd.read_csv("data/covid.csv", dtype={"fips": str})
covid['date'] = pd.to_datetime(covid['date'])
top50 = pd.read_csv("data/top50.csv", dtype={"fips": str})

In [7]:
import plotly.express as px

In [8]:
covid_l = covid[covid["date"] == "2020-09-30"]
covid_l

,date,county,state,fips,cases,deaths
577661,2020-09-30,Autauga,Alabama,1001,1791,27.0
577662,2020-09-30,Baldwin,Alabama,1003,5640,52.0
577663,2020-09-30,Barbour,Alabama,1005,896,7.0
577664,2020-09-30,Bibb,Alabama,1007,664,11.0
577665,2020-09-30,Blount,Alabama,1009,1629,15.0
...,...,...,...,...,...,...
580867,2020-09-30,Sweetwater,Wyoming,56037,339,2.0
580868,2020-09-30,Teton,Wyoming,56039,577,1.0
580869,2020-09-30,Uinta,Wyoming,56041,357,2.0
580870,2020-09-30,Washakie,Wyoming,56043,115,6.0


In [ ]:
#Pull ridership data from Google BigQuery

query = f"""
    SELECT metro_area, area_land, area_water, GEOID10, mode, month, trips, classification, lat, lon
    FROM `transitpolicyapp.ridership.Ridership`
    GROUP BY metro_area, area_land, area_water, GEOID10, mode, month, trips, classification, lat, lon
"""
ridership = bqclient.query(query).to_dataframe()
ridership.head()

In [ ]:
#Generate interactive altair ridership chart by mode and month

search = "Philadelphia, PA-NJ-DE-MD"

ridership_s = ridership[ridership["metro_area"] == search]
chart_ridership = alt.Chart(ridership_s).mark_area().encode(
    x=alt.X('month:T', axis=alt.Axis(title='Month')),
    y=alt.Y('trips:Q', axis=alt.Axis(title='Total Transit Trips')),
    color="mode:N",
    tooltip=['month','mode', 'trips']
).interactive()
chart_ridership.save('chart_ridership.html')

In [ ]:
top50_c = top50.merge(covid_l, on="fips", how='left')
top50_c.head()

In [ ]:
select_date = "2020-09"
ridership_agg = ridership[ridership["month"] == select_date]
ridership_agg = ridership_agg.groupby(['GEOID10', 'metro_area'], as_index=False).agg({'trips': 'sum', 'lat': 'min', 'lon': 'min'})
ridership_agg.head()

In [ ]:
ridership_agg = ridership_agg[['GEOID10','trips']]
top50_c = top50_c.merge(ridership_agg, on="GEOID10", how='left')
top50_c.head()

In [ ]:
#Plot relationship between covid cases and transit trips

chart_scatter = alt.Chart(top50_c).mark_circle(size=200).encode(
    x=alt.X('cases', axis=alt.Axis(title='COVID-19 Cases (Cumulative)')),
    y=alt.Y('trips', axis=alt.Axis(title='Total Transit Trips in September 2020')),
    color=alt.Color('state', legend=None),
    tooltip=['metro_area', 'cases', 'deaths', 'trips']
).interactive()
chart_scatter.save('scatter.html')

In [ ]:
#Query covid data for a sample city

search_county = "San Francisco"
covid_s = covid[covid["county"] == search_county]
covid_s['new_cases'] = covid_s['cases'].diff()
covid_s['new_deaths'] = covid_s['deaths'].diff()
covid_s['cases_avg'] = covid_s.iloc[:,6].rolling(window=7).mean()
covid_s['deaths_avg'] = covid_s.iloc[:,7].rolling(window=7).mean()
covid_s['DateTime'] = pd.to_datetime(covid_s['date'])
covid_s

In [ ]:
#Plot covid cases with altair

chart_cases = alt.Chart(covid_s).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Month')),
    y=alt.Y('cases_avg:Q', axis=alt.Axis(title='New COVID-19 Cases Reported')),
)
chart_cases.save('chart_cases.html')

In [ ]:
#Plot covid deaths with altair

chart_deaths = alt.Chart(covid_s).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Month')),
    y=alt.Y('deaths_avg:Q', axis=alt.Axis(title='New COVID-19 Deaths Reported')),
    color=alt.value('orange'),
)
chart_deaths.save('chart_deaths.html')

In [ ]:
fig = [px.scatter_mapbox(ridership_agg,
                        lat="lat",
                        lon="lon",
                        size="trips",
                        # https://plotly.com/python/builtin-colorscales/
                        color_continuous_scale=px.colors.sequential.Jet,
                        size_max=80,
                        zoom=3,
                        hover_data=["trips"],
                        hover_name='metro_area'
                        ),
       px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
      ]
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(
    title=f'Total Transit Trips in {select_date}',
)
fig.show()
fig.write_html("map.html")